In [83]:
import pandas as pd
import numpy as np

My goal was to solve a constrained optimization; find the minimium variance portfolio subject to the sum of the weights being one and all of the weights being positive. To do so I followed the procedure outlined in Constrained Optimization and Lagrange Multiplier Methods, by Dimitri P Bertsekas.

   That is if one wants to solve a contrained optimization problem, then they can do so by preforming gradient descent on the two norm of the gradient of the orginal problem, and the square of the constraint mathematically, if we want to minimize, 
    $$ L(x,\lambda) = f(x) - \lambda h(x)$$
where f(x) is the objective and h(x) is the constraint, gradient descent won't work because lambda is linear and thus is a saddle point and not an extrema. However we can minimize,
    $$ G(x,\lambda) = \frac{||{\nabla_x L}||^2}{2} + \frac{h(x)^2}{2}$$
which will provide relative minimas (or unfortunately maybe maximas) for our original problem. The derivation of G will be given bellow.

In [19]:
#reading in the data
df = pd.read_csv('4600_COV.txt')

In [76]:
#data is given in a weird format just getting it into a way that is better to work with
data = {'MS':{}}
index = 'MS'
for item in df.values:
    temp = item[0].split('   ')
    if len(temp) == 1:
        index = temp[0]
        data[index] = {}
    else:
        data[index][temp[1]] =temp[0] 


In [77]:
cov = pd.DataFrame(data)

In [119]:
cov = cov.reindex(cov.columns.values)
for x in data.keys():
    for y in data[x].keys():
        cov.loc[x,y]= np.float64(data[x][y])


In [120]:
cov = cov.reindex(cov.columns.values)

In [125]:
cov

,MS,RTN,OXY,TWX,F,AIG,ACN,HON,QCOM,BA,...,FDX,VZ,KO,TGT,WBA,MCD,CVS,KMI,LLY,GD
MS,77.274,10.5038,23.501,18.7331,31.057,31.5852,21.0155,16.5982,24.4191,14.2228,...,26.9671,-10.9368,-3.2989,6.24252,14.6732,5.73627,6.80296,15.0013,1.18838,20.3368
RTN,10.5038,24.2271,-1.68199,5.05844,2.1858,9.30456,8.90474,5.68606,8.89258,10.0658,...,2.44675,3.44848,4.24806,3.46714,8.49319,4.83853,4.93207,-1.75833,-2.10028,11.0958
OXY,23.501,-1.68199,40.1102,7.84004,6.64594,15.5521,6.89018,5.69992,3.65595,6.74503,...,4.88645,1.33458,1.9777,-3.96181,4.76558,5.3496,3.81373,12.0434,-0.398238,4.7249
TWX,18.7331,5.05844,7.84004,40.4981,7.48962,13.6146,15.8629,8.76381,10.597,10.311,...,10.0437,7.72373,5.05675,11.0481,15.6778,7.93723,10.2187,9.31589,1.32585,6.72577
F,31.057,2.1858,6.64594,7.48962,41.9048,16.6947,9.44063,12.1178,14.3309,14.3405,...,15.1454,-4.01772,1.48568,7.00723,12.4109,4.01048,6.57687,6.64123,-2.09904,10.0488
AIG,31.5852,9.30456,15.5521,13.6146,16.6947,39.5721,15.3109,11.1613,13.8193,13.14,...,14.0746,-1.04288,6.52046,4.07557,11.3538,7.52402,7.54195,5.14017,0.0255798,8.74551
ACN,21.0155,8.90474,6.89018,15.8629,9.44063,15.3109,27.3009,10.9407,13.3969,6.6695,...,10.3515,3.5953,8.23562,7.10133,14.2387,10.0546,10.0994,6.26114,4.8823,5.62564
HON,16.5982,5.68606,5.69992,8.76381,12.1178,11.1613,10.9407,12.7752,10.1154,9.55806,...,10.0469,2.94406,4.41806,5.43349,13.6502,7.12302,7.82223,6.51922,2.75052,6.14591
QCOM,24.4191,8.89258,3.65595,10.597,14.3309,13.8193,13.3969,10.1154,48.2643,6.48662,...,14.4661,0.219922,4.51363,4.19979,14.872,4.63262,6.86685,16.4065,-0.784845,9.71726
BA,14.2228,10.0658,6.74503,10.311,14.3405,13.14,6.6695,9.55806,6.48662,33.3315,...,13.7704,3.4004,8.47479,8.5392,13.776,7.68364,7.885,-3.64489,0.653657,8.01829


Frist we define,
    $$ L(w,\lambda) = \phi(w)^T \Sigma \phi(w) - \lambda(e^T \phi(w) - 1)$$
where w is a vector of the weights, $\phi(w)$ is ReLu applied element wise to w, $\Sigma$ is the covariance matrix, and e is a vector of ones the size of w. In order to simplify notation we let $u = \phi(w)$
    $$ \nabla_u L = \Sigma u - \lambda e $$
    $$ \nabla_w L = (\Sigma u - \lambda e) \odot H(w) $$
Where $H(w)$ is the heaviside step function applied element wise to w.
    $$ ||\nabla_w L||^2 = \sum_{i=1}^n  \frac{((\Sigma u - \lambda e)_i H(w_i))^2}{2}$$
let $ q = (\Sigma u - \lambda e)$
    $$ ||\nabla_w L||^2 \frac{\partial}{\partial w_j}  = \sum_{i=1}^n  ((q)_i H(w_i)) \frac{\partial q_i}{\partial w_j}$$
    $$ \frac{\partial q_i}{\partial w_j} = \sum_{s = 1}^n \Sigma_{is} H(w_s) \delta_{sj}$$
    $$ \frac{\partial q_i}{\partial w_j} = \Sigma_{ij} H(w_j)$$
    $$ ||\nabla_w L||^2 \frac{\partial}{\partial w_j}  = \sum_{i=1}^n  ((q)_i H(w_i)) \Sigma_{ij} H(w_j) $$
Now define 
    $$ p(w,\lambda) = ||\nabla_w L||^2$$ 
then,
    $$ \nabla_w p = \Sigma((\Sigma u - \lambda e) \odot H(w)) \odot H(w)$$
    $$ \frac{\partial p}{\partial \lambda} = -((\Sigma u - \lambda e) \odot H(w))^T e$$
    
Now define 
    $$ h(w) = \frac{(u^Te)^2 -2(u^Te)+1}{2} $$
then,
    $$ \nabla_w h = ((u^Te)-1)H(w)$$
    
Now we start with a random w, and $\lambda$, and itteratively compute the gradient and L and update w and $\lambda$ as follows.
    $$ w_n = w_{n-1} - \eta (\nabla_{w_{n-1}} h + \nabla_{w_{n-1}} p)$$
    $$ \lambda_n = \lambda_{n-1} - \eta \frac{\partial p}{\partial \lambda_{n-1}}$$
    
Where $\eta$ is the learning rate.


In [190]:
#defining a rule function 
relu = lambda x: np.maximum(x,0)


In [336]:
w = np.random.rand(50)
epsilon = 1e-7
max_itters = 1e7
i = 0
learning_rate = .000005
check_freq = 10000
vel1 = np.zeros(50)
vel2 = 0
k = np.random.rand()
mu = .3
ones = np.ones(50)
while i< max_itters and np.abs(vel1).mean()<epsilon:
    i+=1
    u = relu(w)
    A = (cov.values.astype(float).dot(u)-k)
    vel2 = mu*vel2 + learning_rate*(-A*np.heaviside(w,0)).dot(ones)
    vel1 = mu*vel1 + learning_rate*(cov.values.astype(float).dot(A*np.heaviside(w,0))+u.T.dot(ones)-1)*np.heaviside(w,0)
    k-= + vel2
    w-=vel1
    if i%check_freq==0:
        print(u.sum(),(u/u.sum()).T.dot(cov.values.astype(float).dot(u/u.sum())))
    

1.0001192904911043 5.1554637742072185
1.000119258815936 5.1554637742072185
1.0001192271491788 5.155463774207217
1.00011919549083 5.155463774207216
1.0001191638408873 5.155463774207215
1.0001191321993486 5.155463774207213
1.0001191005662116 5.155463774207213
1.000119068941474 5.155463774207214
1.0001190373251343 5.155463774207212
1.0001190057171896 5.15546377420721


In [332]:
pd.DataFrame(u,index=cov.columns,columns=['Weights'])

,Weights
MS,0.000000
RTN,0.161038
OXY,0.000000
TWX,0.000000
F,0.000000
AIG,0.000000
ACN,0.000000
HON,0.000000
QCOM,0.000000
BA,0.000000
